In [1]:
# =============================================================================
# CELL 1: Install dependencies
# =============================================================================
import sys
import importlib.util
import subprocess

try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "uv", "--quiet"])
except Exception:
    pass

def install_package_uv(package_list):
    """Install packages using uv in the current kernel."""
    cmd = [
        sys.executable, "-m", "uv", "pip", "install",
        "--python", sys.executable, "--quiet"
    ] + package_list
    subprocess.check_call(cmd)

print("🔍 Checking environment and dependencies...")

requirements = [
    "pandas", "numpy", "plotly>=5.0.0", "ipyfilechooser", "cmocean",
    "xarray", "netCDF4",
    "ipywidgets==8.1.1", "jupyterlab_widgets==3.0.9"
]

pkgs_installed = False
packages_to_install = []

for req in requirements:
    pkg_name = req.split('>')[0].split('=')[0].split('<')[0]
    if "widgets" in pkg_name:
        packages_to_install.append(req)
        continue
    if importlib.util.find_spec(pkg_name) is None:
        print(f"📦 Missing: {pkg_name}")
        packages_to_install.append(req)

if packages_to_install:
    print(f"🚀 Installing {len(packages_to_install)} packages...")
    try:
        install_package_uv(packages_to_install)
        pkgs_installed = True
    except subprocess.CalledProcessError as e:
        print(f"❌ Error: {e}")

try:
    import google.colab
    from google.colab import output
    output.enable_custom_widget_manager()
    print("✅ Google Colab mode configured.")
except ImportError:
    pass

print("\n" + "="*50)
if pkgs_installed:
    print("⚠️ RESTART KERNEL to apply changes.")
else:
    print("✅ All dependencies installed.")
print("="*50)

🔍 Checking environment and dependencies...
🚀 Installing 2 packages...

⚠️ RESTART KERNEL to apply changes.


In [1]:
"""
Trawling4PACE Explorer - v3.0
==============================
Features:
- Day-level filtering (year/month/day)
- Physical field overlay from NetCDF (FSLE, SST, etc.)
- Auto-detect field subdirectories
- Smart date matching (specific date = auto-match field)
- FSLE display fixes (replace inf with NaN, robust colorscale)
- Multiple data layers with individual transparency
- CMOcean and Plotly colorscales

Author: Leandro (USP/IEAPM)
Project: 2026-proj-Trawling4PACE / NASA PACE Hackweek 2026
"""

from __future__ import annotations

import os
import re
import glob
import threading
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Any, Tuple
from pathlib import Path
from datetime import datetime, date

import numpy as np
import pandas as pd
import xarray as xr
import plotly.graph_objects as go
import plotly

import ipywidgets as widgets
from ipyfilechooser import FileChooser
from IPython.display import display, clear_output

# Try to import cmocean
try:
    import cmocean
    HAS_CMOCEAN = True
except ImportError:
    HAS_CMOCEAN = False
    print("[WARNING] cmocean not installed")


# ============================================================
# COLORSCALE DEFINITIONS
# ============================================================

PLOTLY_COLORSCALES = [
    "Viridis", "Plasma", "Inferno", "Magma", "Cividis",
    "Turbo", "Hot", "Jet", "Rainbow",
    "Blues", "Greens", "Reds", "Oranges", "Purples",
    "YlOrRd", "YlGnBu", "RdBu", "RdYlBu", "RdYlGn"
]

CMOCEAN_COLORSCALES = [
    "thermal", "haline", "solar", "ice", "gray", "oxy", "deep",
    "dense", "algae", "matter", "turbid", "speed", "amp", "tempo",
    "rain", "phase", "topo", "balance", "delta", "curl", "diff", "tarn"
] if HAS_CMOCEAN else []


def get_cmocean_colorscale(name: str) -> List[List]:
    """Convert cmocean colormap to Plotly format."""
    if not HAS_CMOCEAN:
        return "Viridis"
    try:
        cmap = getattr(cmocean.cm, name)
        colors = cmap(np.linspace(0, 1, 256))
        colorscale = []
        for i, c in enumerate(colors):
            pos = i / (len(colors) - 1)
            rgb = f"rgb({int(c[0]*255)},{int(c[1]*255)},{int(c[2]*255)})"
            colorscale.append([pos, rgb])
        return colorscale
    except Exception:
        return "Viridis"


# ============================================================
# PLOTLY API COMPATIBILITY
# ============================================================

def _get_plotly_version() -> tuple:
    try:
        return tuple(int(p) for p in plotly.__version__.split('.')[:3])
    except Exception:
        return (0, 0, 0)

_USE_NEW_API = _get_plotly_version() >= (5, 24, 0) and hasattr(go, "Scattermap")


def make_scatter_map(**kwargs):
    return go.Scattermap(**kwargs) if _USE_NEW_API else go.Scattermapbox(**kwargs)


def make_density_map(**kwargs):
    return go.Densitymap(**kwargs) if _USE_NEW_API else go.Densitymapbox(**kwargs)


def apply_map_layout(fig, style, center_lat, center_lon, zoom):
    safe_styles = ["open-street-map", "carto-positron", "carto-darkmatter"]
    if style not in safe_styles:
        style = "open-street-map"
    layout_map = dict(style=style, center=dict(lat=center_lat, lon=center_lon), zoom=zoom)
    if _USE_NEW_API:
        fig.update_layout(map=layout_map)
    else:
        fig.update_layout(mapbox=layout_map)


# ============================================================
# COLUMN DETECTION UTILITIES
# ============================================================

def _norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "", str(s).lower())


def guess_lat_lon_columns(cols):
    ncols = {c: _norm(c) for c in cols}
    lat_cands = [c for c in cols if "lat" in ncols[c]]
    lon_cands = [c for c in cols if "lon" in ncols[c]]
    def score(c):
        nc = ncols[c]
        s = 0
        if "decdeg" in nc: s += 50
        if "beg" in nc: s += 20
        return s
    lat = sorted(lat_cands, key=score, reverse=True)[0] if lat_cands else None
    lon = sorted(lon_cands, key=score, reverse=True)[0] if lon_cands else None
    return lat, lon


def guess_date_column(cols):
    """Auto-detect date column."""
    prefs = ["BEGIN_GMT_TOWDATE", "TOWDATE", "DATE", "DATETIME"]
    for p in prefs:
        if p in cols:
            return p
    for c in cols:
        if "date" in _norm(c) or "time" in _norm(c):
            return c
    return None


def guess_depth_column(cols):
    prefs = ["AVGDEPTH", "SETDEPTH", "ENDDEPTH", "MAXDEPTH", "MINDEPTH"]
    for p in prefs:
        if p in cols:
            return p
    for c in cols:
        if "depth" in _norm(c):
            return c
    return None


def guess_species_column(cols):
    if "SCIENTIFIC_NAME" in cols:
        return "SCIENTIFIC_NAME"
    for c in cols:
        if "scientific" in _norm(c) and "name" in _norm(c):
            return c
    return None


def calculate_bounding_box(df, lat_col, lon_col, margin=2.0):
    lat = pd.to_numeric(df[lat_col], errors="coerce")
    lon = pd.to_numeric(df[lon_col], errors="coerce")
    mask = lat.notna() & lon.notna()
    if mask.sum() == 0:
        return 38.0, -73.0, 5.0
    lat_min, lat_max = lat[mask].min() - margin, lat[mask].max() + margin
    lon_min, lon_max = lon[mask].min() - margin, lon[mask].max() + margin
    center_lat = (lat_min + lat_max) / 2
    center_lon = (lon_min + lon_max) / 2
    max_range = max(lat_max - lat_min, lon_max - lon_min)
    if max_range > 40: zoom = 3.0
    elif max_range > 20: zoom = 4.0
    elif max_range > 10: zoom = 5.0
    elif max_range > 5: zoom = 6.0
    elif max_range > 2: zoom = 7.0
    else: zoom = 8.0
    return center_lat, center_lon, zoom


# ============================================================
# FIELD DATA MANAGER (NetCDF)
# ============================================================

class FieldDataManager:
    """
    Manages physical field data from NetCDF files in subdirectories.
    
    Expected structure:
        data/
        ├── filtered_bts.csv
        ├── fsle/
        │   ├── fsle_20240307.nc
        │   └── ...
        ├── sst/
        │   └── ...
    """
    
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.field_dirs = {}  # {name: Path}
        self.available_dates = {}  # {name: [dates]}
        self.variables = {}  # {name: [var_names]}
        self._cache = {}  # {(name, date): xr.Dataset}
    
    def scan_directories(self) -> Dict[str, bool]:
        """
        Scan data directory for subdirectories containing NetCDF files.
        
        Returns
        -------
        dict
            {subdir_name: has_netcdf_files}
        """
        result = {}
        if not self.data_dir.exists():
            return result
        
        for item in self.data_dir.iterdir():
            if item.is_dir() and not item.name.startswith('.'):
                # Check for NetCDF files
                nc_files = list(item.glob('*.nc'))
                result[item.name] = len(nc_files) > 0
        
        return result
    
    def register_field_dir(self, name: str):
        """
        Register a subdirectory as a field data source.
        Parses available dates and variables from files.
        """
        dir_path = self.data_dir / name
        if not dir_path.exists():
            return
        
        self.field_dirs[name] = dir_path
        
        # Parse dates from filenames (format: prefix_YYYYMMDD.nc)
        dates = []
        nc_files = sorted(dir_path.glob('*.nc'))
        
        for f in nc_files:
            # Try to extract date from filename
            match = re.search(r'(\d{8})', f.stem)
            if match:
                try:
                    d = datetime.strptime(match.group(1), '%Y%m%d').date()
                    dates.append(d)
                except:
                    pass
        
        self.available_dates[name] = sorted(set(dates))
        
        # Get variables from first file
        if nc_files:
            try:
                with xr.open_dataset(nc_files[0]) as ds:
                    # Exclude coordinate variables
                    coords = set(ds.coords.keys())
                    vars = [v for v in ds.data_vars if v not in coords and v != 'crs']
                    self.variables[name] = vars
            except Exception as e:
                print(f"[WARNING] Could not read variables from {nc_files[0]}: {e}")
                self.variables[name] = []
    
    def unregister_field_dir(self, name: str):
        """Remove a field directory from registration."""
        self.field_dirs.pop(name, None)
        self.available_dates.pop(name, None)
        self.variables.pop(name, None)
    
    def get_field_data(self, name: str, target_date: date) -> Optional[xr.Dataset]:
        """
        Load field data for a specific date.
        
        Parameters
        ----------
        name : str
            Field directory name (e.g., 'fsle')
        target_date : date
            Target date
            
        Returns
        -------
        xr.Dataset or None
        """
        if name not in self.field_dirs:
            return None
        
        # Check cache
        cache_key = (name, target_date)
        if cache_key in self._cache:
            return self._cache[cache_key]
        
        # Find file for date
        dir_path = self.field_dirs[name]
        date_str = target_date.strftime('%Y%m%d')
        
        # Try common patterns
        patterns = [
            f"*{date_str}*.nc",
            f"*_{date_str}.nc",
            f"{name}_{date_str}.nc"
        ]
        
        for pattern in patterns:
            files = list(dir_path.glob(pattern))
            if files:
                try:
                    ds = xr.open_dataset(files[0])
                    self._cache[cache_key] = ds
                    return ds
                except Exception as e:
                    print(f"[ERROR] Loading {files[0]}: {e}")
                    return None
        
        return None
    
    def find_nearest_date(self, name: str, target_date: date) -> Optional[date]:
        """
        Find the nearest available date for a field.
        """
        if name not in self.available_dates or not self.available_dates[name]:
            return None
        
        dates = self.available_dates[name]
        
        # Exact match
        if target_date in dates:
            return target_date
        
        # Find nearest
        diffs = [(abs((d - target_date).days), d) for d in dates]
        diffs.sort()
        return diffs[0][1] if diffs else None
    
    def get_date_range(self, name: str) -> Tuple[Optional[date], Optional[date]]:
        """Get min/max dates for a field."""
        if name not in self.available_dates or not self.available_dates[name]:
            return None, None
        dates = self.available_dates[name]
        return min(dates), max(dates)


# ============================================================
# LAYER CONFIGURATION
# ============================================================

@dataclass
class LayerConfig:
    name: str = "Layer"
    variable: str = ""
    colorscale: str = "Viridis"
    colorscale_type: str = "plotly"
    opacity: float = 0.8
    scale_type: str = "linear"
    vmin: Optional[float] = None
    vmax: Optional[float] = None
    visible: bool = True
    marker_size: int = 8
    layer_type: str = "scatter"


# ============================================================
# LAYER PANEL WIDGET
# ============================================================

class LayerPanel:
    """Widget panel for configuring a single data layer."""
    
    def __init__(self, layer_id: int, numeric_cols: List[str], on_change_callback=None):
        self.layer_id = layer_id
        self.on_change = on_change_callback
        self._building = True
        
        colorscale_opts = []
        for cs in PLOTLY_COLORSCALES:
            colorscale_opts.append((f"📊 {cs}", f"plotly:{cs}"))
        for cs in CMOCEAN_COLORSCALES:
            colorscale_opts.append((f"🌊 {cs}", f"cmocean:{cs}"))
        
        self.w_enabled = widgets.Checkbox(value=True, description="", 
                                          layout=widgets.Layout(width="30px"), indent=False)
        self.w_variable = widgets.Dropdown(options=["(none)"] + numeric_cols, value="(none)",
                                           description="", layout=widgets.Layout(width="180px"))
        self.w_colorscale = widgets.Dropdown(options=colorscale_opts, value="plotly:Viridis",
                                             description="", layout=widgets.Layout(width="160px"))
        self.w_opacity = widgets.FloatSlider(value=0.8, min=0.1, max=1.0, step=0.05,
                                             description="", readout_format=".2f",
                                             layout=widgets.Layout(width="140px"), continuous_update=False)
        self.w_scale = widgets.ToggleButtons(options=[("Lin", "linear"), ("Log", "log")],
                                             value="linear", description="",
                                             layout=widgets.Layout(width="100px"), style={'button_width': '45px'})
        self.w_size = widgets.IntSlider(value=8, min=3, max=20, description="",
                                        layout=widgets.Layout(width="120px"), continuous_update=False)
        self.w_type = widgets.ToggleButtons(options=[("Pts", "scatter"), ("Den", "density")],
                                            value="scatter", description="",
                                            layout=widgets.Layout(width="100px"), style={'button_width': '45px'})
        
        self._wire_callbacks()
        self._building = False
    
    def _wire_callbacks(self):
        for w in [self.w_enabled, self.w_variable, self.w_colorscale, 
                  self.w_opacity, self.w_scale, self.w_size, self.w_type]:
            w.observe(self._on_widget_change, names="value")
    
    def _on_widget_change(self, change):
        if not self._building and self.on_change:
            self.on_change()
    
    def get_config(self) -> Optional[LayerConfig]:
        if not self.w_enabled.value or self.w_variable.value == "(none)":
            return None
        cs_parts = self.w_colorscale.value.split(":")
        return LayerConfig(
            name=f"L{self.layer_id+1}: {self.w_variable.value}",
            variable=self.w_variable.value,
            colorscale=cs_parts[1],
            colorscale_type=cs_parts[0],
            opacity=self.w_opacity.value,
            scale_type=self.w_scale.value,
            visible=True,
            marker_size=self.w_size.value,
            layer_type=self.w_type.value
        )
    
    def set_defaults(self, variable=None, colorscale=None, scale_type=None, 
                     layer_type=None, opacity=None):
        self._building = True
        if variable and variable in list(self.w_variable.options):
            self.w_variable.value = variable
        if colorscale:
            self.w_colorscale.value = colorscale
        if scale_type:
            self.w_scale.value = scale_type
        if layer_type:
            self.w_type.value = layer_type
        if opacity is not None:
            self.w_opacity.value = opacity
        self._building = False
    
    def set_enabled(self, enabled: bool):
        for w in [self.w_enabled, self.w_variable, self.w_colorscale,
                  self.w_opacity, self.w_scale, self.w_size, self.w_type]:
            w.disabled = not enabled
    
    def get_widget(self) -> widgets.HBox:
        return widgets.HBox([
            self.w_enabled,
            widgets.HTML(f"<b>{self.layer_id+1}</b>", layout=widgets.Layout(width="25px")),
            self.w_variable, self.w_colorscale, self.w_opacity, 
            self.w_scale, self.w_size, self.w_type
        ])


# ============================================================
# MAIN EXPLORER CLASS
# ============================================================

class T4PExplorer:
    """
    Trawling4PACE Explorer v3.0
    
    Interactive map explorer for fisheries data with physical field overlay.
    """
    
    MAX_LAYERS = 4
    
    def __init__(self, start_dir: Optional[str] = None):
        self.start_dir = start_dir or os.getcwd()
        self.df = None
        self.current_file = None
        self.data_dir = None
        self.field_manager = None
        
        # Column references
        self._date_col = None
        self._species_col = None
        self._parsed_dates = None  # pd.Series of parsed dates
        
        # State flags
        self._updating = False
        self._rendering = False
        self._cancel_requested = False
        
        # Layer panels
        self.layer_panels = []
        
        # Build UI
        self._build_ui()
        self._wire_callbacks()
        
        self._debug_info()
    
    def _debug_info(self):
        print("="*50)
        print(f"T4P Explorer v3.0")
        print(f"Plotly: {plotly.__version__}")
        print(f"CMOcean: {HAS_CMOCEAN}")
        print(f"Map API: {'MapLibre' if _USE_NEW_API else 'Mapbox'}")
        print("="*50)
    
    def _build_ui(self):
        """Build the user interface."""
        
        # === HEADER ===
        header = widgets.HTML("""
        <div style="background: linear-gradient(135deg, #0d3b66 0%, #1a5f7a 100%); 
                    padding: 15px; border-radius: 8px; margin-bottom: 10px;">
            <h2 style="color: white; margin: 0;">🌊 Trawling4PACE Explorer v3.0</h2>
            <p style="color: #ccc; margin: 5px 0 0 0;">NASA PACE Hackweek 2026</p>
        </div>
        """)
        
        # === LOADING ===
        self.loading_indicator = widgets.HTML(value="", layout=widgets.Layout(width="300px"))
        self.btn_cancel = widgets.Button(description="Cancel", icon="stop", button_style="danger",
                                         layout=widgets.Layout(width="100px", display="none"))
        self.progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description="",
                                                  bar_style="info",
                                                  layout=widgets.Layout(width="200px", display="none"))
        
        # === FILE SELECTION ===
        self.fc = FileChooser(self.start_dir)
        self.fc.filter_pattern = "*.csv"
        self.fc.title = "Select a CSV file"
        self.fc.use_dir_icons = True
        
        self.btn_load = widgets.Button(description="Load CSV", icon="upload",
                                       button_style="success", layout=widgets.Layout(width="120px"))
        self.btn_autozoom = widgets.Button(description="Auto-zoom", icon="crosshairs",
                                           layout=widgets.Layout(width="110px"))
        self.btn_refresh = widgets.Button(description="Refresh", icon="refresh",
                                          button_style="primary", layout=widgets.Layout(width="100px"))
        self.status = widgets.HTML("<b>Status:</b> Select a CSV file and click Load")
        
        # === COLUMN MAPPING ===
        self.lat_dd = widgets.Dropdown(description="Latitude:", options=[],
                                       layout=widgets.Layout(width="320px"))
        self.lon_dd = widgets.Dropdown(description="Longitude:", options=[],
                                       layout=widgets.Layout(width="320px"))
        
        # === FILTERS ===
        self.species_dd = widgets.Dropdown(description="Species:", options=["(all)"],
                                           layout=widgets.Layout(width="450px"))
        
        # Date filter mode
        self.date_mode = widgets.RadioButtons(
            options=[
                ('Specific Date (Year/Month/Day)', 'specific'),
                ('Date Range (Period)', 'range')
            ],
            value='specific',
            description='Filter:',
            layout=widgets.Layout(width='350px')
        )
        
        # Specific date picker
        self.date_picker = widgets.DatePicker(
            description='Date:',
            value=None,
            layout=widgets.Layout(width='250px')
        )
        
        # Date range
        self.date_start = widgets.DatePicker(description='From:', value=None,
                                              layout=widgets.Layout(width='200px'))
        self.date_end = widgets.DatePicker(description='To:', value=None,
                                            layout=widgets.Layout(width='200px'))
        self.date_range_box = widgets.HBox([self.date_start, self.date_end])
        
        # Date container (switches based on mode)
        self.date_container = widgets.VBox([self.date_picker])
        
        # Depth
        self.depth_dd = widgets.Dropdown(description="Depth Col:", options=["(none)"],
                                         layout=widgets.Layout(width="280px"))
        self.depth_rs = widgets.FloatRangeSlider(description="Depth (m):", min=0, max=1000,
                                                  value=(0, 1000), continuous_update=False,
                                                  layout=widgets.Layout(width="450px"))
        
        # === FIELD DIRECTORIES ===
        self.field_dir_container = widgets.VBox([])
        self.field_checkboxes = {}  # {name: Checkbox}
        
        # Field date override (for range mode)
        self.field_date_override = widgets.DatePicker(
            description='Field Date:',
            value=None,
            layout=widgets.Layout(width='250px')
        )
        self.field_date_label = widgets.HTML(
            value="<i>Auto-matched from filter</i>",
            layout=widgets.Layout(width='200px')
        )
        self.field_date_box = widgets.HBox([
            self.field_date_override,
            self.field_date_label
        ])
        
        # Field variable selector
        self.field_source_dd = widgets.Dropdown(
            description='Source:',
            options=[],
            layout=widgets.Layout(width='200px')
        )
        self.field_var_dd = widgets.Dropdown(
            description='Variable:',
            options=[],
            layout=widgets.Layout(width='200px')
        )
        self.field_colorscale_dd = widgets.Dropdown(
            description='Colors:',
            options=[(f"🌊 {cs}", cs) for cs in CMOCEAN_COLORSCALES] + 
                    [(f"📊 {cs}", cs) for cs in PLOTLY_COLORSCALES],
            value='amp' if 'amp' in CMOCEAN_COLORSCALES else 'Viridis',
            layout=widgets.Layout(width='180px')
        )
        self.field_opacity = widgets.FloatSlider(
            description='Opacity:',
            value=0.6, min=0.1, max=1.0, step=0.05,
            continuous_update=False,
            layout=widgets.Layout(width='250px')
        )
        self.field_enabled = widgets.Checkbox(
            value=False,
            description='Show field overlay',
            layout=widgets.Layout(width='200px')
        )
        
        # === MAP SETTINGS ===
        self.basemap_dd = widgets.Dropdown(
            description="Basemap:",
            options=[
                ("OpenStreetMap", "open-street-map"),
                ("Carto Positron (Light)", "carto-positron"),
                ("Carto Dark", "carto-darkmatter")
            ],
            value="carto-positron",
            layout=widgets.Layout(width="280px")
        )
        self.zoom_slider = widgets.FloatSlider(description="Zoom:", min=2, max=12, step=0.5, value=5.0,
                                               continuous_update=False, layout=widgets.Layout(width="280px"))
        self.center_lat = widgets.FloatText(description="Center Lat:", value=38.0,
                                            layout=widgets.Layout(width="160px"))
        self.center_lon = widgets.FloatText(description="Center Lon:", value=-73.0,
                                            layout=widgets.Layout(width="160px"))
        
        # === LAYER CONFIGURATION ===
        self.layer_container = widgets.VBox([])
        
        layer_header = widgets.HBox([
            widgets.HTML("<b>On</b>", layout=widgets.Layout(width="30px")),
            widgets.HTML("<b>#</b>", layout=widgets.Layout(width="25px")),
            widgets.HTML("<b>Variable</b>", layout=widgets.Layout(width="180px")),
            widgets.HTML("<b>Colorscale</b>", layout=widgets.Layout(width="160px")),
            widgets.HTML("<b>Opacity</b>", layout=widgets.Layout(width="140px")),
            widgets.HTML("<b>Scale</b>", layout=widgets.Layout(width="100px")),
            widgets.HTML("<b>Size</b>", layout=widgets.Layout(width="120px")),
            widgets.HTML("<b>Type</b>", layout=widgets.Layout(width="100px")),
        ], layout=widgets.Layout(margin="5px 0", padding="5px", background_color="#f0f0f0"))
        
        # === OUTPUTS ===
        self.out_fig = widgets.Output(layout=widgets.Layout(width="100%", height="620px",
                                                            border="1px solid #ddd"))
        self.out_log = widgets.Output(layout=widgets.Layout(width="100%", height="150px",
                                                            overflow="auto"))
        
        # === TABS ===
        tab_data = widgets.VBox([
            widgets.HTML("<h4>📁 File Selection</h4>"),
            self.fc,
            widgets.HBox([self.btn_load, self.btn_refresh, self.btn_autozoom]),
            self.status,
            widgets.HTML("<hr><h4>🗂️ Coordinate Columns</h4>"),
            widgets.HBox([self.lat_dd, self.lon_dd])
        ])
        
        tab_filter = widgets.VBox([
            widgets.HTML("<h4>🔍 Data Filters</h4>"),
            self.species_dd,
            widgets.HTML("<hr><h4>📅 Date Filter</h4>"),
            self.date_mode,
            self.date_container,
            widgets.HTML("<hr>"),
            self.depth_dd,
            self.depth_rs
        ])
        
        tab_fields = widgets.VBox([
            widgets.HTML("<h4>🗺️ Physical Field Overlay</h4>"),
            widgets.HTML("""<p style='color:#666; font-size:12px;'>
            Select subdirectories containing NetCDF field data (FSLE, SST, etc.).<br>
            The field date will auto-match your filter or can be overridden.
            </p>"""),
            widgets.HTML("<b>Available field directories:</b>"),
            self.field_dir_container,
            widgets.HTML("<hr>"),
            self.field_enabled,
            widgets.HBox([self.field_source_dd, self.field_var_dd]),
            widgets.HBox([self.field_colorscale_dd, self.field_opacity]),
            widgets.HTML("<b>Field date:</b>"),
            self.field_date_box
        ])
        
        tab_layers = widgets.VBox([
            widgets.HTML("<h4>📊 CSV Data Layers</h4>"),
            widgets.HTML("""<p style='color:#666; font-size:12px;'>
            Configure up to 4 layers. 🌊 = CMOcean, 📊 = Plotly
            </p>"""),
            layer_header,
            self.layer_container
        ])
        
        tab_map = widgets.VBox([
            widgets.HTML("<h4>🗺️ Map Settings</h4>"),
            self.basemap_dd,
            self.zoom_slider,
            widgets.HBox([self.center_lat, self.center_lon])
        ])
        
        self.tabs = widgets.Tab()
        self.tabs.children = [tab_data, tab_filter, tab_fields, tab_layers, tab_map]
        self.tabs.set_title(0, "📂 Data")
        self.tabs.set_title(1, "🔍 Filters")
        self.tabs.set_title(2, "🗺️ Fields")
        self.tabs.set_title(3, "📊 Layers")
        self.tabs.set_title(4, "⚙️ Map")
        
        # === MAIN LAYOUT ===
        controls = widgets.VBox([
            self.tabs,
            widgets.HBox([self.loading_indicator, self.btn_cancel, self.progress_bar]),
            widgets.HTML("<hr><b>📋 Log:</b>"),
            self.out_log
        ], layout=widgets.Layout(width="500px"))
        
        self.main_layout = widgets.VBox([
            header,
            widgets.HBox([controls, self.out_fig])
        ])
    
    def _wire_callbacks(self):
        """Connect all widget callbacks."""
        self.btn_load.on_click(lambda _: self.load_file())
        self.btn_autozoom.on_click(lambda _: self.auto_zoom())
        self.btn_refresh.on_click(lambda _: self.render())
        self.btn_cancel.on_click(lambda _: self._request_cancel())
        
        # Date mode toggle
        self.date_mode.observe(self._on_date_mode_change, names='value')
        
        # Field source change
        self.field_source_dd.observe(self._on_field_source_change, names='value')
        
        # Reactive widgets
        reactive = [
            self.lat_dd, self.lon_dd, self.species_dd,
            self.date_picker, self.date_start, self.date_end,
            self.depth_dd, self.depth_rs,
            self.basemap_dd, self.zoom_slider, self.center_lat, self.center_lon,
            self.field_enabled, self.field_source_dd, self.field_var_dd,
            self.field_colorscale_dd, self.field_opacity, self.field_date_override
        ]
        for w in reactive:
            w.observe(self._on_reactive_change, names="value")
    
    def _on_date_mode_change(self, change):
        """Handle date filter mode change."""
        if change['new'] == 'specific':
            self.date_container.children = [self.date_picker]
            self.field_date_label.value = "<i>Auto-matched from filter</i>"
        else:
            self.date_container.children = [self.date_range_box]
            self.field_date_label.value = "<i>Override for period (middle date used)</i>"
        
        if not self._updating:
            self.render()
    
    def _on_field_source_change(self, change):
        """Update variable options when field source changes."""
        if self.field_manager and change['new']:
            vars = self.field_manager.variables.get(change['new'], [])
            self.field_var_dd.options = vars if vars else ['(none)']
            if vars:
                self.field_var_dd.value = vars[0]
    
    def _on_reactive_change(self, change):
        if not self._updating and not self._rendering and self.df is not None:
            self.render()
    
    def _request_cancel(self):
        self._cancel_requested = True
        self.loading_indicator.value = "<span style='color:orange'>⏳ Cancelling...</span>"
    
    def _set_controls_enabled(self, enabled: bool):
        """Enable or disable all controls."""
        for w in [self.btn_load, self.btn_autozoom, self.btn_refresh,
                  self.lat_dd, self.lon_dd, self.species_dd,
                  self.date_picker, self.date_start, self.date_end,
                  self.depth_dd, self.depth_rs, self.basemap_dd,
                  self.zoom_slider, self.center_lat, self.center_lon,
                  self.field_enabled, self.field_source_dd, self.field_var_dd,
                  self.field_colorscale_dd, self.field_opacity, self.field_date_override]:
            w.disabled = not enabled
        for panel in self.layer_panels:
            panel.set_enabled(enabled)
        self.btn_cancel.layout.display = "none" if enabled else "inline-block"
        self.progress_bar.layout.display = "none" if enabled else "inline-block"
    
    def _show_loading(self, message: str = "Processing..."):
        self._rendering = True
        self._cancel_requested = False
        self.loading_indicator.value = f"<span style='color:#0066cc'>⏳ {message}</span>"
        self._set_controls_enabled(False)
        self.progress_bar.value = 0
    
    def _hide_loading(self):
        self._rendering = False
        self.loading_indicator.value = ""
        self._set_controls_enabled(True)
    
    def _update_progress(self, value: float, message: str = None):
        self.progress_bar.value = value
        if message:
            self.loading_indicator.value = f"<span style='color:#0066cc'>⏳ {message}</span>"
    
    def _on_layer_change(self):
        if not self._updating and not self._rendering and self.df is not None:
            self.render()
    
    def _on_field_checkbox_change(self, change):
        """Handle field directory checkbox change."""
        if self.field_manager is None:
            return
        
        # Find which checkbox changed
        for name, cb in self.field_checkboxes.items():
            if cb.value:
                if name not in self.field_manager.field_dirs:
                    self.field_manager.register_field_dir(name)
            else:
                self.field_manager.unregister_field_dir(name)
        
        # Update source dropdown
        sources = list(self.field_manager.field_dirs.keys())
        self.field_source_dd.options = sources if sources else ['(none)']
        if sources:
            self.field_source_dd.value = sources[0]
    
    def _setup_field_directories(self):
        """Scan and setup field directory checkboxes."""
        if self.data_dir is None:
            return
        
        self.field_manager = FieldDataManager(self.data_dir)
        subdirs = self.field_manager.scan_directories()
        
        if not subdirs:
            self.field_dir_container.children = [
                widgets.HTML("<i>No subdirectories found</i>")
            ]
            return
        
        checkboxes = []
        self.field_checkboxes = {}
        
        for name, has_nc in subdirs.items():
            cb = widgets.Checkbox(
                value=has_nc,  # Auto-check if has NetCDF files
                description=f"{name}/ {'✅' if has_nc else '⚠️ no .nc files'}",
                disabled=not has_nc,
                layout=widgets.Layout(width='300px')
            )
            cb.observe(self._on_field_checkbox_change, names='value')
            self.field_checkboxes[name] = cb
            checkboxes.append(cb)
            
            # Auto-register if has files
            if has_nc:
                self.field_manager.register_field_dir(name)
        
        self.field_dir_container.children = checkboxes
        
        # Update source dropdown
        sources = list(self.field_manager.field_dirs.keys())
        self.field_source_dd.options = sources if sources else ['(none)']
        if sources:
            self.field_source_dd.value = sources[0]
    
    def _create_layer_panels(self, numeric_cols: List[str]):
        """Create layer configuration panels."""
        self.layer_panels = []
        panel_widgets = []
        for i in range(self.MAX_LAYERS):
            panel = LayerPanel(i, numeric_cols, on_change_callback=self._on_layer_change)
            self.layer_panels.append(panel)
            panel_widgets.append(panel.get_widget())
        self.layer_container.children = panel_widgets
    
    def _set_smart_defaults(self, numeric_cols: List[str]):
        """Set smart default values for layers."""
        if len(self.layer_panels) < 2:
            return
        
        # Layer 1: Temperature
        for col in ["SURFTEMP", "BOTTEMP", "BKTTEMP"]:
            if col in numeric_cols:
                self.layer_panels[0].set_defaults(
                    variable=col, colorscale="cmocean:thermal",
                    scale_type="linear", layer_type="density", opacity=0.6
                )
                break
        
        # Layer 2: Catch weight
        for col in ["EXPCATCHWT", "EXPCATCHNUM"]:
            if col in numeric_cols:
                self.layer_panels[1].set_defaults(
                    variable=col, colorscale="cmocean:haline",
                    scale_type="log", layer_type="scatter", opacity=0.8
                )
                break
    
    def load_file(self):
        """Load selected CSV file."""
        path = self.fc.selected
        if not path or not str(path).lower().endswith(".csv"):
            self.status.value = "<b>Status:</b> ⚠️ Please select a .csv file"
            return
        
        try:
            self._show_loading("Loading file...")
            self._updating = True
            
            with self.out_log:
                print(f"\n[LOAD] {path}")
            
            self._update_progress(20, "Reading CSV...")
            df = pd.read_csv(path, low_memory=False)
            
            if self._cancel_requested:
                self._hide_loading()
                self._updating = False
                return
            
            # Remove unnamed index column
            if df.columns.size > 0 and str(df.columns[0]).startswith("Unnamed"):
                df = df.drop(columns=[df.columns[0]])
            
            self.df = df
            self.current_file = path
            self.data_dir = Path(path).parent
            
            self._update_progress(40, "Detecting columns...")
            self._update_column_widgets()
            
            self._update_progress(50, "Parsing dates...")
            self._parse_dates()
            
            self._update_progress(60, "Setting up filters...")
            self._update_filter_widgets()
            
            self._update_progress(70, "Scanning field directories...")
            self._setup_field_directories()
            
            # Create layer panels
            numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
            self._create_layer_panels(numeric_cols)
            
            self._update_progress(80, "Setting defaults...")
            self._set_smart_defaults(numeric_cols)
            
            # Calculate bounding box
            lat_col, lon_col = self.lat_dd.value, self.lon_dd.value
            if lat_col and lon_col:
                center_lat, center_lon, zoom = calculate_bounding_box(df, lat_col, lon_col, margin=2.0)
                self.center_lat.value = round(center_lat, 2)
                self.center_lon.value = round(center_lon, 2)
                self.zoom_slider.value = zoom
            
            self.status.value = (
                f"<b>Status:</b> ✅ Loaded <code>{os.path.basename(path)}</code> "
                f"({len(df):,} rows, {df.shape[1]} cols)"
            )
            
            with self.out_log:
                print(f"[OK] {len(df):,} rows")
                print(f"[OK] Field dirs: {list(self.field_manager.field_dirs.keys()) if self.field_manager else []}")
            
            self._updating = False
            self._update_progress(100, "Rendering...")
            self._hide_loading()
            self.render()
            
        except Exception as e:
            self._hide_loading()
            self._updating = False
            self.status.value = f"<b>Status:</b> ❌ Error: {str(e)[:50]}"
            with self.out_log:
                import traceback
                traceback.print_exc()
    
    def _update_column_widgets(self):
        """Update column selection dropdowns."""
        if self.df is None:
            return
        
        cols = list(self.df.columns)
        lat_guess, lon_guess = guess_lat_lon_columns(cols)
        depth_guess = guess_depth_column(cols)
        self._species_col = guess_species_column(cols)
        self._date_col = guess_date_column(cols)
        
        self.lat_dd.options = cols
        self.lon_dd.options = cols
        if lat_guess: self.lat_dd.value = lat_guess
        if lon_guess: self.lon_dd.value = lon_guess
        
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns.tolist()
        self.depth_dd.options = ["(none)"] + numeric_cols
        if depth_guess and depth_guess in numeric_cols:
            self.depth_dd.value = depth_guess
    
    def _parse_dates(self):
        """Parse date column for filtering."""
        if self.df is None or self._date_col is None:
            self._parsed_dates = None
            return
        
        if self._date_col not in self.df.columns:
            self._parsed_dates = None
            return
        
        try:
            self._parsed_dates = pd.to_datetime(self.df[self._date_col], errors='coerce')
            with self.out_log:
                valid = self._parsed_dates.notna().sum()
                print(f"[OK] Parsed {valid:,} valid dates from '{self._date_col}'")
        except Exception as e:
            self._parsed_dates = None
            with self.out_log:
                print(f"[WARNING] Could not parse dates: {e}")
    
    def _update_filter_widgets(self):
        """Update filter widgets based on data."""
        if self.df is None:
            return
        
        df = self.df
        
        # Species filter
        if self._species_col and self._species_col in df.columns:
            species_list = sorted(df[self._species_col].dropna().astype(str).unique().tolist())
            self.species_dd.options = ["(all)"] + species_list
        else:
            self.species_dd.options = ["(all)"]
        
        # Date range from parsed dates
        if self._parsed_dates is not None:
            valid_dates = self._parsed_dates.dropna()
            if len(valid_dates) > 0:
                min_date = valid_dates.min().date()
                max_date = valid_dates.max().date()
                self.date_picker.value = max_date
                self.date_start.value = min_date
                self.date_end.value = max_date
        
        # Depth range
        depth_col = self.depth_dd.value
        if depth_col and depth_col != "(none)" and depth_col in df.columns:
            d = pd.to_numeric(df[depth_col], errors="coerce").dropna()
            if len(d) > 0:
                lo, hi = float(d.min()), float(d.max())
                if np.isfinite(lo) and np.isfinite(hi) and hi > lo:
                    self.depth_rs.min = np.floor(lo)
                    self.depth_rs.max = np.ceil(hi)
                    self.depth_rs.value = (self.depth_rs.min, self.depth_rs.max)
    
    def _get_filtered_data(self) -> pd.DataFrame:
        """Apply all filters and return filtered DataFrame."""
        if self.df is None:
            return pd.DataFrame()
        
        df = self.df
        lat_col, lon_col = self.lat_dd.value, self.lon_dd.value
        
        if lat_col not in df.columns or lon_col not in df.columns:
            return pd.DataFrame()
        
        # Valid coordinates
        out = df[df[lat_col].notna() & df[lon_col].notna()].copy()
        
        # Species filter
        if self._species_col and self._species_col in out.columns and self.species_dd.value != "(all)":
            out = out[out[self._species_col].astype(str) == str(self.species_dd.value)]
        
        # Date filter
        if self._parsed_dates is not None and self._date_col in out.columns:
            dates = pd.to_datetime(out[self._date_col], errors='coerce')
            
            if self.date_mode.value == 'specific':
                # Specific date filter
                if self.date_picker.value:
                    target = pd.Timestamp(self.date_picker.value)
                    out = out[dates.dt.date == target.date()]
            else:
                # Date range filter
                if self.date_start.value and self.date_end.value:
                    start = pd.Timestamp(self.date_start.value)
                    end = pd.Timestamp(self.date_end.value)
                    out = out[(dates >= start) & (dates <= end)]
        
        # Depth filter
        depth_col = self.depth_dd.value
        if depth_col and depth_col != "(none)" and depth_col in out.columns:
            d = pd.to_numeric(out[depth_col], errors="coerce")
            lo, hi = self.depth_rs.value
            out = out[(d >= lo) & (d <= hi)]
        
        return out
    
    def _get_field_date(self) -> Optional[date]:
        """
        Determine which date to use for field data.
        
        - Specific date mode: use the selected date
        - Range mode: use override if set, otherwise middle of range
        """
        if self.date_mode.value == 'specific':
            return self.date_picker.value
        else:
            # Check for override
            if self.field_date_override.value:
                return self.field_date_override.value
            
            # Calculate middle of range
            if self.date_start.value and self.date_end.value:
                start = self.date_start.value
                end = self.date_end.value
                middle = start + (end - start) / 2
                return middle
            
            return None
    
    def _get_colorscale(self, config: LayerConfig):
        """Get colorscale for a layer config."""
        if config.colorscale_type == "cmocean":
            return get_cmocean_colorscale(config.colorscale)
        return config.colorscale
    
    def _build_figure(self, df: pd.DataFrame) -> go.Figure:
        """Build the Plotly figure."""
        fig = go.Figure()
        lat_col, lon_col = self.lat_dd.value, self.lon_dd.value
        colorbar_x = 1.02
        
        # === FIELD OVERLAY ===
        if self.field_enabled.value and self.field_manager:
            try:
                field_name = self.field_source_dd.value
                field_var = self.field_var_dd.value
                field_date = self._get_field_date()
                
                if field_name and field_var and field_date:
                    # Find nearest available date
                    actual_date = self.field_manager.find_nearest_date(field_name, field_date)
                    
                    if actual_date:
                        ds = self.field_manager.get_field_data(field_name, actual_date)
                        
                        if ds is not None and field_var in ds.data_vars:
                            data = ds[field_var]
                            
                            # Select first time step if multiple
                            if 'time' in data.dims and len(data.time) > 1:
                                data = data.isel(time=0)
                            elif 'time' in data.dims:
                                data = data.isel(time=0)
                            
                            # FSLE FIX: Replace inf with NaN
                            data = data.where(np.isfinite(data))
                            
                            # Get coordinates
                            lats = data.lat.values if 'lat' in data.coords else data.latitude.values
                            lons = data.lon.values if 'lon' in data.coords else data.longitude.values
                            
                            # ROBUST: Calculate vmin/vmax ignoring outliers
                            values = data.values.flatten()
                            values = values[np.isfinite(values)]
                            if len(values) > 0:
                                vmin = np.percentile(values, 2)
                                vmax = np.percentile(values, 98)
                            else:
                                vmin, vmax = 0, 1
                            
                            # Get colorscale
                            cs_name = self.field_colorscale_dd.value
                            if cs_name in CMOCEAN_COLORSCALES:
                                colorscale = get_cmocean_colorscale(cs_name)
                            else:
                                colorscale = cs_name
                            
                            # Create heatmap trace
                            # Note: Plotly Scattermap doesn't support true raster overlay
                            # We use a contour approximation
                            lon_grid, lat_grid = np.meshgrid(lons, lats)
                            
                            # Subsample for performance
                            step = max(1, len(lats) // 100)
                            
                            lat_sub = lat_grid[::step, ::step].flatten()
                            lon_sub = lon_grid[::step, ::step].flatten()
                            z_sub = data.values[::step, ::step].flatten()
                            
                            # Filter valid values
                            mask = np.isfinite(z_sub)
                            
                            if mask.sum() > 0:
                                trace = make_density_map(
                                    name=f"{field_name}: {field_var} ({actual_date})",
                                    lat=lat_sub[mask],
                                    lon=lon_sub[mask],
                                    z=z_sub[mask],
                                    radius=15,
                                    opacity=self.field_opacity.value,
                                    colorscale=colorscale,
                                    zmin=vmin,
                                    zmax=vmax,
                                    showscale=True,
                                    colorbar=dict(
                                        title=f"{field_var}<br>({actual_date})",
                                        x=colorbar_x,
                                        len=0.4,
                                        y=0.2
                                    )
                                )
                                fig.add_trace(trace)
                                colorbar_x += 0.08
                                
                                with self.out_log:
                                    print(f"[FIELD] {field_name}/{field_var} @ {actual_date}")
            except Exception as e:
                with self.out_log:
                    print(f"[FIELD ERROR] {e}")
        
        # === CSV LAYERS ===
        for i, panel in enumerate(self.layer_panels):
            config = panel.get_config()
            if config is None or config.variable not in df.columns:
                continue
            
            z = pd.to_numeric(df[config.variable], errors="coerce")
            mask = z.notna()
            if mask.sum() == 0:
                continue
            
            z_display = z.copy()
            if config.scale_type == "log":
                z_display = np.log10(z_display.clip(lower=1e-10))
            
            colorscale = self._get_colorscale(config)
            
            if config.layer_type == "density":
                trace = make_density_map(
                    name=config.name,
                    lat=df.loc[mask, lat_col],
                    lon=df.loc[mask, lon_col],
                    z=z_display.loc[mask],
                    radius=config.marker_size + 5,
                    opacity=config.opacity,
                    colorscale=colorscale,
                    showscale=True,
                    colorbar=dict(
                        title=f"{config.variable}<br>({'log' if config.scale_type == 'log' else 'lin'})",
                        x=colorbar_x,
                        len=0.4,
                        y=0.8 - (i * 0.25)
                    )
                )
            else:
                hover_text = df.loc[mask].apply(
                    lambda r: f"<b>{config.variable}</b>: {r[config.variable]:.3f}<br>"
                              f"Lat: {r[lat_col]:.4f}, Lon: {r[lon_col]:.4f}",
                    axis=1
                )
                trace = make_scatter_map(
                    name=config.name,
                    mode="markers",
                    lat=df.loc[mask, lat_col],
                    lon=df.loc[mask, lon_col],
                    marker=dict(
                        size=config.marker_size,
                        opacity=config.opacity,
                        color=z_display.loc[mask],
                        colorscale=colorscale,
                        showscale=True,
                        colorbar=dict(
                            title=f"{config.variable}<br>({'log' if config.scale_type == 'log' else 'lin'})",
                            x=colorbar_x,
                            len=0.4,
                            y=0.8 - (i * 0.25)
                        )
                    ),
                    text=hover_text,
                    hoverinfo="text"
                )
            
            fig.add_trace(trace)
            colorbar_x += 0.08
        
        # Layout
        fig.update_layout(
            height=600,
            margin=dict(l=0, r=100, t=50, b=0),
            title=dict(
                text=f"T4P | {os.path.basename(self.current_file) if self.current_file else 'N/A'} | N={len(df):,}",
                x=0.5,
                font=dict(size=14)
            ),
            legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01, bgcolor="rgba(255,255,255,0.8)")
        )
        
        apply_map_layout(fig, self.basemap_dd.value, float(self.center_lat.value),
                         float(self.center_lon.value), float(self.zoom_slider.value))
        
        return fig
    
    def render(self):
        """Render the map."""
        if self._rendering:
            return
        
        with self.out_fig:
            clear_output(wait=True)
            
            if self.df is None:
                display(widgets.HTML("""
                <div style="padding: 60px; text-align: center; color: #666;">
                    <h3>📂 Load a CSV file to visualize</h3>
                </div>
                """))
                return
            
            try:
                self._show_loading("Filtering data...")
                df_f = self._get_filtered_data()
                
                if self._cancel_requested:
                    self._hide_loading()
                    return
                
                if len(df_f) == 0:
                    self._hide_loading()
                    display(widgets.HTML("""
                    <div style="padding: 40px; text-align: center; color: #c00;">
                        <h3>⚠️ No data after filtering</h3>
                        <p>Adjust filters or date selection.</p>
                    </div>
                    """))
                    return
                
                active_layers = [p.get_config() for p in self.layer_panels if p.get_config()]
                
                if not active_layers and not self.field_enabled.value:
                    self._hide_loading()
                    display(widgets.HTML("""
                    <div style="padding: 40px; text-align: center; color: #666;">
                        <h3>📊 No layers configured</h3>
                        <p>Configure CSV layers or enable field overlay.</p>
                    </div>
                    """))
                    return
                
                self._update_progress(50, f"Building map ({len(df_f):,} points)...")
                fig = self._build_figure(df_f)
                
                if self._cancel_requested:
                    self._hide_loading()
                    return
                
                self._update_progress(90, "Rendering...")
                fig.show()
                self._hide_loading()
                
                with self.out_log:
                    print(f"[RENDER] OK - {len(df_f):,} pts, {len(active_layers)} layers")
                
            except Exception as e:
                self._hide_loading()
                display(widgets.HTML(f"<div style='color:#c00;'><h3>❌ Error</h3><p>{e}</p></div>"))
                with self.out_log:
                    import traceback
                    traceback.print_exc()
    
    def auto_zoom(self):
        """Auto-zoom to data extent."""
        if self.df is None:
            return
        self._updating = True
        df_f = self._get_filtered_data()
        if len(df_f) == 0:
            self._updating = False
            return
        lat_col, lon_col = self.lat_dd.value, self.lon_dd.value
        center_lat, center_lon, zoom = calculate_bounding_box(df_f, lat_col, lon_col, margin=2.0)
        self.center_lat.value = round(center_lat, 2)
        self.center_lon.value = round(center_lon, 2)
        self.zoom_slider.value = zoom
        self._updating = False
        self.render()
    
    def show(self):
        """Display the explorer."""
        display(self.main_layout)


# ============================================================
# FACTORY & AUTO-RUN
# ============================================================

def create_explorer(start_dir: Optional[str] = None) -> T4PExplorer:
    return T4PExplorer(start_dir=start_dir)

T4PApp = T4PExplorer
create_app = create_explorer

if __name__ == "__main__":
    app = create_explorer()
    app.show()

T4P Explorer v3.0
Plotly: 6.5.2
CMOcean: True
Map API: MapLibre


In [6]:
import pandas as pd
import os

# 1. Locate the data file
# We check multiple common paths to avoid 'File Not Found' errors
possible_paths = [
    "filtered_bts.csv",
    "../data/filtered_bts.csv",
    "../../data/filtered_bts.csv",
    "/home/desenvolvedor/projetos/hackweek/2026-proj-Trawling4PACE/data/filtered_bts.csv"
]

file_path = None
for path in possible_paths:
    if os.path.exists(path):
        file_path = path
        print(f"📂 Reading data from: {file_path}")
        break

if file_path is None:
    print("❌ Error: File 'filtered_bts.csv' not found.")
else:
    # 2. Load necessary columns
    # We load Date AND Coordinates to calculate spatial extent
    # Adjust column names if your CSV uses different ones
    cols_to_load = ['BEGIN_GMT_TOWDATE', 'DECDEG_BEGLAT', 'DECDEG_BEGLON']
    
    try:
        # Read the CSV
        df = pd.read_csv(file_path, usecols=cols_to_load)

        # 3. Process Dates
        # errors='coerce' turns invalid parsing into NaT (Not a Time)
        dates = pd.to_datetime(df['BEGIN_GMT_TOWDATE'], errors='coerce')
        dates = dates.dropna()
        
        # Extract unique dates (YYYY-MM-DD only) and sort them
        unique_days = sorted(dates.dt.date.unique())
        
        # 4. Process Coordinates
        # Ensure values are numeric
        lats = pd.to_numeric(df['DECDEG_BEGLAT'], errors='coerce').dropna()
        lons = pd.to_numeric(df['DECDEG_BEGLON'], errors='coerce').dropna()

        # 5. Save dates to a TXT file
        output_filename = "unique_dates_found.txt"
        
        with open(output_filename, "w") as f:
            f.write(f"Unique Dates List (Total: {len(unique_days)})\n")
            f.write("="*30 + "\n")
            for day in unique_days:
                f.write(f"{day}\n")
                
        # 6. Display Statistics (in English)
        print("\n" + "="*50)
        print("📊 DATASET SUMMARY & STATISTICS")
        print("="*50)
        
        if not lats.empty and not lons.empty:
            print("🌎 GEOGRAPHIC EXTENT (Bounding Box):")
            print(f"   Latitude:  {lats.min():.4f}  to  {lats.max():.4f}")
            print(f"   Longitude: {lons.min():.4f}  to  {lons.max():.4f}")
        else:
            print("⚠️ Geographic columns found but contain no valid data.")

        print("-" * 50)
        
        if unique_days:
            # Fixed the string literal below to ensure no syntax errors
            print("📅 TEMPORAL COVERAGE:")
            print(f"   Start Date: {unique_days[0]}")
            print(f"   End Date:   {unique_days[-1]}")
            print(f"   Total Active Days: {len(unique_days)}")
            
            print("-" * 50)
            print(f"✅ SUCCESS! Date list saved to file:")
            print(f"📄 {output_filename}")
        else:
            print("⚠️ No valid dates found in the file.")
            
        print("="*50)

    except ValueError as e:
        print(f"❌ Error reading columns. Check if {cols_to_load} exist in your CSV.")
        print(f"Details: {e}")

📂 Reading data from: ../../data/filtered_bts.csv

📊 DATASET SUMMARY & STATISTICS
🌎 GEOGRAPHIC EXTENT (Bounding Box):
   Latitude:  34.4170  to  44.4229
   Longitude: -76.0774  to  -65.7390
--------------------------------------------------
📅 TEMPORAL COVERAGE:
   Start Date: 2024-03-07
   End Date:   2025-05-15
   Total Active Days: 153
--------------------------------------------------
✅ SUCCESS! Date list saved to file:
📄 unique_dates_found.txt
